In [1]:
import numpy as np
import xarray as xr

# modules for plotting datetime data
import matplotlib.dates as mdates
from matplotlib.axis import Axis

# modules for using datetime variables
import datetime
from datetime import time

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm

from xgcm import Grid
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import cartopy.crs as ccrs
import cmocean

import subprocess as sp

import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.ticker import ScalarFormatter

from xclim import ensembles
from xoverturning import calcmoc
import cmip_basins

import cftime
from pandas.errors import OutOfBoundsDatetime  # Import the specific error

In [2]:
myVars = globals()

In [3]:
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/read_and_calculate.ipynb # to get selecting_basins function
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/plotting_functions.ipynb
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/read_mem1_functions.ipynb
%run /home/Kiera.Lowman/Kd-sensitivity-analysis/notebooks/custom_moc_functions.ipynb

# Test reading mem2 and mem3 data

In [4]:
avg_period = 25

mem1_start = 176
mem1_end = 200

mem2_start = 376
mem2_end = 400

mem3_start = 576
mem3_end = 600

In [5]:
test_ctrl_mem1 = get_pp_av_data('tune_ctrl_1860IC_200yr',mem1_start,mem1_end,avg_period,
                               pp_type='av-annual')#,debug=True)
test_ctrl_mem2 = get_pp_av_data('tune_ctrl_1860IC_201-2001',mem2_start,mem2_end,avg_period,
                               pp_type='av-annual',time_decoding=False)#,debug=True)
test_ctrl_mem3 = get_pp_av_data('tune_ctrl_1860IC_201-2001',mem3_start,mem3_end,avg_period,
                               pp_type='av-annual',time_decoding=False)#,debug=True)
print(test_ctrl_mem1.time)
print(test_ctrl_mem2.time)
print(test_ctrl_mem3.time)

<xarray.DataArray 'time' (time: 1)> Size: 8B
array([cftime.DatetimeNoLeap(188, 7, 2, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 8B 0188-07-02 12:00:00
Attributes:
    long_name:       time
    cartesian_axis:  T
    calendar_type:   NOLEAP
    bounds:          time_bnds
<xarray.DataArray 'time' (time: 1)> Size: 8B
array([cftime.DatetimeNoLeap(388, 7, 2, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 8B 0388-07-02 12:00:00
<xarray.DataArray 'time' (time: 1)> Size: 8B
array([cftime.DatetimeNoLeap(588, 7, 2, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 8B 0588-07-02 12:00:00


In [6]:
test_surf_mem1 = get_pp_av_data('surf_1860IC_200yr_0.5TW',mem1_start,mem1_end,avg_period,
                               pp_type='av-annual')
test_surf_mem1.time.values

array([cftime.DatetimeNoLeap(188, 7, 2, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)

In [7]:
test_surf_mem2 = get_pp_av_data('mem2_surf_1860IC_200yr_0.5TW',mem2_start,mem2_end,avg_period,
                               pp_type='av-annual',time_decoding=False)
test_surf_mem2.time.values

array([cftime.DatetimeNoLeap(388, 7, 2, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)

In [8]:
test_surf_mem3 = get_pp_av_data('mem3_surf_1860IC_200yr_0.5TW',mem3_start,mem3_end,avg_period,
                               pp_type='av-annual',time_decoding=False)
test_surf_mem3.time.values

array([cftime.DatetimeNoLeap(588, 7, 2, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)

In [9]:
var_to_diff = ["temp", "N2", "age", "rhopot2", "salt"]

mem1_diff = test_surf_mem1[var_to_diff] - test_ctrl_mem1[var_to_diff]
mem1_diff

<xarray.Dataset> Size: 52MB
Dimensions:  (xh: 360, yh: 210, z_l: 34, z_i: 35, time: 1)
Coordinates:
  * xh       (xh) float64 3kB -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yh       (yh) float64 2kB -76.75 -76.25 -75.75 -75.25 ... 88.75 89.25 89.75
  * z_l      (z_l) float64 272B 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03
  * z_i      (z_i) float64 280B 0.0 5.0 15.0 25.0 ... 5.25e+03 5.75e+03 6.25e+03
  * time     (time) object 8B 0188-07-02 12:00:00
    geolon   (yh, xh) float32 302kB -279.5 -278.5 -277.5 ... 79.95 79.97 79.99
    geolat   (yh, xh) float32 302kB -76.75 -76.75 -76.75 ... 66.01 65.6 65.18
Data variables:
    temp     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    N2       (time, z_i, yh, xh) float32 11MB dask.array<chunksize=(1, 35, 210, 360), meta=np.ndarray>
    age      (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    rhopot2  (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    salt     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>

In [10]:
mem2_diff = test_surf_mem2[var_to_diff] - test_ctrl_mem2[var_to_diff]
mem2_diff

<xarray.Dataset> Size: 52MB
Dimensions:  (yh: 210, z_l: 34, z_i: 35, time: 1, xh: 360)
Coordinates:
  * yh       (yh) float64 2kB -76.75 -76.25 -75.75 -75.25 ... 88.75 89.25 89.75
  * z_l      (z_l) float64 272B 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03
  * z_i      (z_i) float64 280B 0.0 5.0 15.0 25.0 ... 5.25e+03 5.75e+03 6.25e+03
  * time     (time) object 8B 0388-07-02 12:00:00
  * xh       (xh) float64 3kB -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
    geolon   (yh, xh) float32 302kB -279.5 -278.5 -277.5 ... 79.95 79.97 79.99
    geolat   (yh, xh) float32 302kB -76.75 -76.75 -76.75 ... 66.01 65.6 65.18
Data variables:
    temp     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    N2       (time, z_i, yh, xh) float32 11MB dask.array<chunksize=(1, 35, 210, 360), meta=np.ndarray>
    age      (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    rhopot2  (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    salt     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>

In [11]:
mem3_diff = test_surf_mem3[var_to_diff] - test_ctrl_mem3[var_to_diff]
mem3_diff

<xarray.Dataset> Size: 52MB
Dimensions:  (xh: 360, yh: 210, z_l: 34, z_i: 35, time: 1)
Coordinates:
  * xh       (xh) float64 3kB -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yh       (yh) float64 2kB -76.75 -76.25 -75.75 -75.25 ... 88.75 89.25 89.75
  * z_l      (z_l) float64 272B 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03
  * z_i      (z_i) float64 280B 0.0 5.0 15.0 25.0 ... 5.25e+03 5.75e+03 6.25e+03
  * time     (time) object 8B 0588-07-02 12:00:00
    geolon   (yh, xh) float32 302kB -279.5 -278.5 -277.5 ... 79.95 79.97 79.99
    geolat   (yh, xh) float32 302kB -76.75 -76.75 -76.75 ... 66.01 65.6 65.18
Data variables:
    temp     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    N2       (time, z_i, yh, xh) float32 11MB dask.array<chunksize=(1, 35, 210, 360), meta=np.ndarray>
    age      (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    rhopot2  (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    salt     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>

In [12]:
print(test_ctrl_mem1.time.values)
print(test_surf_mem1.time.values)
print(test_ctrl_mem2.time.values)
print(test_surf_mem2.time.values)
print(test_ctrl_mem3.time.values)
print(test_surf_mem3.time.values)

[cftime.DatetimeNoLeap(188, 7, 2, 12, 0, 0, 0, has_year_zero=True)]
[cftime.DatetimeNoLeap(188, 7, 2, 12, 0, 0, 0, has_year_zero=True)]
[cftime.DatetimeNoLeap(388, 7, 2, 12, 0, 0, 0, has_year_zero=True)]
[cftime.DatetimeNoLeap(388, 7, 2, 12, 0, 0, 0, has_year_zero=True)]
[cftime.DatetimeNoLeap(588, 7, 2, 12, 0, 0, 0, has_year_zero=True)]
[cftime.DatetimeNoLeap(588, 7, 2, 12, 0, 0, 0, has_year_zero=True)]


In [15]:
diff_ens = ensembles.create_ensemble([mem1_diff,mem2_diff,mem3_diff])

diff_ens_mean = ensembles.ensemble_mean_std_max_min(diff_ens)
diff_ens_mean['wet'] = test_ctrl_mem1['wet']
diff_ens_mean['dxt'] = test_ctrl_mem1['dxt']
diff_ens_mean['dyt'] = test_ctrl_mem1['dyt']
diff_ens_mean['areacello'] = test_ctrl_mem1['areacello']#.isel(realization=0)

for var in var_to_diff:
    mean_var_name = var + '_mean'
    diff_ens_mean = diff_ens_mean.rename({mean_var_name: var})

diff_ens_mean

<xarray.Dataset> Size: 622MB
Dimensions:        (xh: 360, yh: 210, z_l: 34, time: 3, z_i: 35)
Coordinates:
  * xh             (xh) float64 3kB -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * yh             (yh) float64 2kB -76.75 -76.25 -75.75 ... 88.75 89.25 89.75
  * z_l            (z_l) float64 272B 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03
  * time           (time) object 24B 0188-07-02 12:00:00 ... 0588-07-02 12:00:00
    geolon         (yh, xh) float32 302kB -279.5 -278.5 -277.5 ... 79.97 79.99
    geolat         (yh, xh) float32 302kB -76.75 -76.75 -76.75 ... 65.6 65.18
  * z_i            (z_i) float64 280B 0.0 5.0 15.0 ... 5.75e+03 6.25e+03
Data variables: (12/24)
    temp           (time, z_l, yh, xh) float32 31MB dask.array<chunksize=(3, 34, 210, 360), meta=np.ndarray>
    temp_stdev     (time, z_l, yh, xh) float32 31MB dask.array<chunksize=(3, 34, 210, 360), meta=np.ndarray>
    temp_max       (time, z_l, yh, xh) float32 31MB dask.array<chunksize=(3, 34, 210, 360), meta=np.ndarray>
    temp_min       (time, z_l, yh, xh) float32 31MB dask.array<chunksize=(3, 34, 210, 360), meta=np.ndarray>
    N2             (time, z_i, yh, xh) float32 32MB dask.array<chunksize=(3, 35, 210, 360), meta=np.ndarray>
    N2_stdev       (time, z_i, yh, xh) float32 32MB dask.array<chunksize=(3, 35, 210, 360), meta=np.ndarray>
    ...             ...
    salt_max       (time, z_l, yh, xh) float32 31MB dask.array<chunksize=(3, 34, 210, 360), meta=np.ndarray>
    salt_min       (time, z_l, yh, xh) float32 31MB dask.array<chunksize=(3, 34, 210, 360), meta=np.ndarray>
    wet            (yh, xh) float32 302kB ...
    dxt            (yh, xh) float32 302kB ...
    dyt            (yh, xh) float32 302kB ...
    areacello      (yh, xh) float32 302kB ...
Attributes:
    history:  [2025-02-05 14:29:16] : Computation of statistics on 3 ensemble...

In [9]:
for i in range(len(test_surf_mem1.time.values)):
    test_surf_mem2.time.values[i] = test_surf_mem1.time.values[i]
    test_surf_mem3.time.values[i] = test_surf_mem1.time.values[i]
    
case_ens = ensembles.create_ensemble([test_surf_mem1,test_surf_mem2,test_surf_mem3])

case_ens_mean = ensembles.ensemble_mean_std_max_min(case_ens[var_list])
case_ens_mean['wet'] = case_ens['wet'].isel(realization=0)
case_ens_mean['dxt'] = case_ens['dxt'].isel(realization=0)
case_ens_mean['dyt'] = case_ens['dyt'].isel(realization=0)
case_ens_mean['areacello'] = case_ens['areacello'].isel(realization=0)

for var in var_list:
    mean_var_name = var + '_mean'
    case_ens_mean = case_ens_mean.rename({mean_var_name: var})

case_ens_mean

<xarray.Dataset> Size: 209MB
Dimensions:        (xh: 360, yh: 210, z_l: 34, time: 1, z_i: 35)
Coordinates:
  * xh             (xh) float64 3kB -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * yh             (yh) float64 2kB -76.75 -76.25 -75.75 ... 88.75 89.25 89.75
  * z_l            (z_l) float64 272B 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03
  * time           (time) object 8B 0188-07-02 12:00:00
    geolon         (yh, xh) float32 302kB -279.5 -278.5 -277.5 ... 79.97 79.99
    geolat         (yh, xh) float32 302kB -76.75 -76.75 -76.75 ... 65.6 65.18
  * z_i            (z_i) float64 280B 0.0 5.0 15.0 ... 5.75e+03 6.25e+03
    realization    int64 8B 0
Data variables: (12/24)
    temp           (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    temp_stdev     (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    temp_max       (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    temp_min       (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    N2             (time, z_i, yh, xh) float32 11MB dask.array<chunksize=(1, 35, 210, 360), meta=np.ndarray>
    N2_stdev       (time, z_i, yh, xh) float32 11MB dask.array<chunksize=(1, 35, 210, 360), meta=np.ndarray>
    ...             ...
    salt_max       (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    salt_min       (time, z_l, yh, xh) float32 10MB dask.array<chunksize=(1, 34, 210, 360), meta=np.ndarray>
    wet            (yh, xh) float32 302kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    dxt            (yh, xh) float32 302kB 2.549e+04 2.549e+04 ... 4.667e+04
    dyt            (yh, xh) float32 302kB 5.56e+04 5.56e+04 ... 2.71e+03 917.4
    areacello      (yh, xh) float32 302kB 1.417e+09 1.417e+09 ... 4.262e+07
Attributes:
    filename:          ocean_monthly_z.0176-0200.ann.nc
    title:             surf_1860IC_200yr_0.5TW
    associated_files:  area_t: 01760101.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A
    comment:           FMS time averaging, version 3.0, precision=double
    history:           [2025-02-05 14:16:06] : Computation of statistics on 3...

In [ ]:
# exp_list is a list of strings of the experiment names

def get_pp_ensemble(exp_list,var,start_yr,end_yr,avg_period):
    ens_mem_array = []
    
    for i in range(len(exp_list)):
        ens_mem_array.append(get_pp_av_data(exp_list[i], start_yr, end_yr, avg_period, pp_type='av'))
        
    case_ens = ensembles.create_ensemble(ens_mem_array)

    case_ens_mean = ensembles.ensemble_mean_std_max_min(case_ens[[var]])
    case_ens_mean['wet'] = case_ens['wet'].isel(realization=0)
    case_ens_mean['dxt'] = case_ens['dxt'].isel(realization=0)
    case_ens_mean['dyt'] = case_ens['dyt'].isel(realization=0)
    case_ens_mean['areacello'] = case_ens['areacello'].isel(realization=0)
    mean_var_name = var+'_mean'
    case_ens_mean = case_ens_mean.rename({mean_var_name: var})

    return case_ens_mean